In [1]:
# Make sure we can load all the packages
try:
    import sys
    import os
    print(os.getcwd())
    import solaris
    sys.path.append("../CosmiQ_SN6_Baseline/")
    import baseline
    sys.path.append("../")
    import spacenet6
    from spacenet6.cosmiq import baseline_wrap
    print("Loaded all packages.")
except:
    raise Exception("Problem loading packages")

/Users/gwilliams/Projects/FractalsProject/spacenet6challenge/notebooks


/Users/gwilliams/opt/miniconda_for_spacenet6_local_jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gwilliams/opt/miniconda_for_spacenet6_local_jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gwilliams/opt/miniconda_for_spacenet6_local_jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

Loaded all packages.


In [ ]:
# Train on SAR only
import os
import datetime
import solaris as sol
import model


# validate sessios toplevel
train_sessions_path = '../../data/spacenet6/train_sessions'
if not os.path.exists(train_sessions_path):
  raise Exception("train sessions path does not exist.")

# create a dir for this training session
dpath = str(datetime.datetime.now()).replace(":","_").replace(" ","_")
outpath = '%s/%s' % ( train_sessions_path, dpath )
os.makedirs(outpath,exist_ok=True)
weightpath = os.path.join(outpath, "weights")
os.makedirs(outpath,exist_ok=True)

# create train.csv from template
f = open('../configs/train.csv')
txt = f.read()
f.close()
newtxt = txt.replace('./root','../../data/spacenet6/pretrain')
traincsv = os.path.join( outpath, "train.csv")
f = open(traincsv, 'w')
f.write(newtxt)
f.flush()
f.close()

# create valid.csv from template
f = open('../configs/valid.csv')
txt = f.read()
f.close()
newtxt = txt.replace('./root','../../data/spacenet6/pretrain')
validcsv = os.path.join( outpath, "valid.csv")
f = open(validcsv, 'w')
f.write(newtxt)
f.flush()
f.close()

# create a SAR training yaml from template
f = open('../configs/train_sar.yaml')
txt = f.read()
f.close()
newtxt = txt.replace("DATEDIR",outpath)
newtxt = newtxt.replace("TRAINCSV",traincsv)
newtxt = newtxt.replace("VALIDCSV",validcsv)
newyaml = os.path.join( outpath, "train_sar.yaml")
f = open(newyaml,'w')
f.write(newtxt)
f.flush()
f.close()

print("Wrote file->%s" % newyaml)

# create a training config from yaml
config = sol.utils.config.parse(newyaml)
config['pretrained'] = False

sar_dict = {
    'model_name': 'unet11',
    'weight_path': None,
    'weight_url': None,
    'arch': model.UNet11
}

# train
trainer = sol.nets.train.Trainer(config, custom_model_dict=sar_dict)
trainer.train()

Wrote file->../../data/spacenet6/train_sessions/2020-05-05_11_32_41.373366/train_sar.yaml
Beginning training epoch 0
Train Pytorch CUDA NOTAVAIL


/Users/distiller/project/conda/conda-bld/pytorch_1587428061935/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


    loss at batch 0: 3.376142978668213
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
    loss at batch 10: 2.023456573486328
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
    loss at batch 20: 1.625366449356079
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
    loss at batch 30: 1.136474847793579
T

In [2]:

resume = False

# Verify all the directories
import os
import datetime

train_path = "../../data/spacenet6/train/AOI_11_Rotterdam"
if not os.path.exists(train_path):
  raise Exception("Could not find training data")

pretrain_path = '../../data/spacenet6/pretrain'
if not os.path.exists(pretrain_path):
  raise Exception('Could not find pretrain data')

train_sessions_path = '../../data/spacenet6/train_sessions'

# are we trying to resume a training session 
if resume: 

  # Get the last trained model and figure out where to resume...
  print("Will try to resume training from trainining=>", resume)
  dpath = resume
  outpath = '%s/%s' % ( train_sessions_path, dpath )
  if not os.path.exists( outpath ):
    raise Exception("Can't resume because %s does not exist." % outpath)
  subdirs = os.listdir( os.path.join(outpath, "weights"))
  print("listdir=", subdirs)
  epochs = sorted( [ int( subdir.split(".")[0] ) for subdir in subdirs if subdir.split(".")[0].isnumeric() ] )
  print("all epochs=", epochs)
  resume_epoch = int(epochs[-1])
  resume_weights = os.path.join(outpath,"weights/%d.hdf5" % resume_epoch )
  raise Exception("Not finished.")
else:

  # Create new dated directory for saved weights...
  resume_epoch = None
  resume_weights = None
  dpath = str(datetime.datetime.now()).replace(":","_").replace(" ","_")
  outpath = '%s/%s' % ( train_sessions_path, dpath )

os.makedirs(outpath, exist_ok=True)
print("model directory=", outpath)

weights_dir = os.path.join( outpath, "weights")
os.makedirs(weights_dir, exist_ok=True)
print("model weights directory=", weights_dir)


model directory= ../../data/spacenet6/train_sessions/2020-05-03_10_40_14.989557
model weights directory= ../../data/spacenet6/train_sessions/2020-05-03_10_40_14.989557/weights


In [ ]:
# Start the training session

cmdargs = ["--train",
                  "--sardir","%s/SAR-Intensity" % train_path,
                  "--opticaldir","%s/PS-RGB" % train_path,
                  "--labeldir","%s/geojson_buildings" % train_path,
                  "--rotationfile","%s/SummaryData/SAR_orientations.txt" % train_path,
                             "--rotationfilelocal","%s/SAR_orientations.txt" % pretrain_path,
                              "--maskdir", "%s/masks" % pretrain_path,
                              "--sarprocdir", "%s/sartrain" % pretrain_path, 
                              "--opticalprocdir","%s/optical" % pretrain_path,
                              "--traincsv","%s/train.csv" % pretrain_path,
                              "--validcsv", "%s/valid.csv" % pretrain_path,
                              "--opticaltraincsv", "%s/opticaltrain.csv" % pretrain_path,
                              "--opticalvalidcsv", "%s/opticalvalid.csv" % pretrain_path,
                              "--testcsv","%s/test.csv" % pretrain_path,
                              "--yamlpath","%s/sar.yaml" % outpath,
                              "--opticalyamlpath","%s/optical.yaml" % outpath,
                              "--modeldir","%s/weights" % outpath,
                              "--testprocdir","%s/sartest" % outpath,
                              "--testoutdir","%s/inference_continuous" % outpath,
                              "--testbinarydir","%s/inference_binary" % outpath,
                              "--testvectordir","%s/inference_vectors" % outpath,
                              "--rotate",
                              "--transferoptical",
                              "--mintrainsize","20",
                              "--mintestsize","80"]
args = baseline_wrap.parse_args(cmdargs)
#print(args)
#import importlib
#importlib.reload(baseline_wrap)
baseline_wrap.invoke(outpath, '../../data/spacenet6/pretrain', args)

For arg 'pretrain', checking if 'False' is a path...
For arg 'train', checking if 'True' is a path...
For arg 'pretest', checking if 'False' is a path...
For arg 'test', checking if 'False' is a path...
For arg 'eval', checking if 'False' is a path...
For arg 'sardir', checking if '../../data/spacenet6/train/AOI_11_Rotterdam/SAR-Intensity' is a path...
For arg 'opticaldir', checking if '../../data/spacenet6/train/AOI_11_Rotterdam/PS-RGB' is a path...
For arg 'labeldir', checking if '../../data/spacenet6/train/AOI_11_Rotterdam/geojson_buildings' is a path...
For arg 'rotationfile', checking if '../../data/spacenet6/train/AOI_11_Rotterdam/SummaryData/SAR_orientations.txt' is a path...
Found file ../../data/spacenet6/train/AOI_11_Rotterdam/SummaryData/SAR_orientations.txt
For arg 'rotationfilelocal', checking if '../../data/spacenet6/pretrain/SAR_orientations.txt' is a path...
Found file ../../data/spacenet6/pretrain/SAR_orientations.txt
For arg 'maskdir', checking if '../../data/spacenet

/Users/distiller/project/conda/conda-bld/pytorch_1587428061935/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


    loss at batch 0: 2.5635058879852295
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
    loss at batch 10: 1.8316764831542969
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
    loss at batch 20: 1.2072319984436035
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
Train Pytorch CUDA NOTAVAIL
    loss at batch 30: 1.03741979598999

In [1]:
print("done")

done
